In [1]:
import matplotlib.pyplot as plt

from PIL import Image
from scipy.misc import imread
from scipy.misc import imsave
import numpy as np
from scipy.stats import multivariate_normal
import math

In [2]:
## Read image
data_path = './Dataset/hw3_img.jpg'
img = Image.open(data_path) # Image object
image = np.array(img) # Array

# get shape
im_w = image.shape[0]
im_h = image.shape[1]

resize_ratio = 0.1
img_resize = img.resize((int(im_h*resize_ratio), int(im_w*resize_ratio))) # Image object
image_resize = np.array(img_resize) # Array

# get shape
im_w_re = image_resize.shape[0]
im_h_re = image_resize.shape[1]

## Normalize pixel value to 0 - 1
im = image/255.0

## Avoid sigular matrix of Cov_mat
bias = 0.0001


# print(im_w,im_h)
# print(im_w_re,im_h_re)
# print(image.shape)
# print(image_resize.shape)

In [3]:
# setting para.
K_cluster = 20
means = np.zeros((K_cluster,3))
for i in range(5):
    for j in range(3):
        means[i,j] = i
k_ary = np.zeros((im_w_re,im_h_re))

In [ ]:
# kmeans
# for k means
# means = np.zeros((1,3,2))
# means[0,:,1]=np.array([1.0, 1.0, 1.0])


def k_means(k_ary, image, means):    
    # init
    init_means = np.zeros_like(means)
    init_cov = np.zeros((K_cluster,3,3))
    init_pi = np.zeros(K_cluster)  
    
    # cal k-array
    for i in range(im_w_re):
        for j in range(im_h_re):
            dist = np.zeros(K_cluster)
            for k in range(K_cluster):
                dist[k] = np.sum((image[i,j,:]-means[k,:])**2)
            k_ary[i,j] = np.argmin(dist)  

    # start update means
    for k in range(K_cluster):
        k_cluster_list = np.where(k_ary == k)
        k_lst_L = len(k_cluster_list[0])
        
        # cal for pi
        init_pi[k] = float(k_lst_L)/float((im_w_re*im_h_re))
        
        # use for create mean 
        if k_lst_L != 0:
            R = image[:,:,0]
            G = image[:,:,1]
            B = image[:,:,2]
            init_means[k,0] = float(R[k_cluster_list].sum())/k_lst_L
            init_means[k,1] = float(G[k_cluster_list].sum())/k_lst_L
            init_means[k,2] = float(B[k_cluster_list].sum())/k_lst_L
            
        # cal Cov.
        N_k = k_lst_L
        val = init_cov[k,:,:]
        
        for i in range(im_w_re):
            for j in range(im_h_re):
                if k_ary[i,j] == k:
                    val += np.dot((image[i,j,:] - init_means[k,:]).reshape((3,1)), (image[i,j,:] - init_means[k,:]).reshape((1,3)))
        init_cov[k,:,:] = val/N_k

    return k_ary, init_means, init_cov, init_pi


IterN = 100
for i in range(IterN):
    k_ary, means, cov, pi = k_means(k_ary, image_resize, means)

print ('======================================')
print ('kmeans - means = \n',means)
print ('kmeans - cov = \n',cov)
print ('kmeans - pi = \n',pi)

# print(means[0,:])
# print(cov[0,:,:])

/Users/huitinghong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in true_divide


kmeans - means = 
 [[  84.75302663   48.7433414    10.41404358]
 [ 107.34983498   96.87293729   17.14851485]
 [ 226.42043222   64.4891945     9.19056974]
 [ 234.23927393  203.35643564   37.42739274]
 [ 224.40940941  227.47147147  212.09209209]
 [ 210.41964966  191.13099772  154.12338157]
 [ 214.44754098  162.39508197   70.66393443]
 [  86.89265537   78.68361582   56.81920904]
 [ 172.69461078   50.61676647   11.93413174]
 [  48.18292683  111.97560976  205.67073171]
 [  55.48590022   41.5010846     4.09978308]
 [ 124.39837398   22.4796748     4.69918699]
 [ 165.42610837  135.38669951   44.43349754]
 [  45.54205607   46.37383178   41.19626168]
 [  69.92857143   11.35714286    2.17142857]
 [ 153.43566591  154.45823928  124.06320542]
 [  38.10364683   27.87332054    3.63915547]
 [  21.68269231   12.31456044    2.42445055]
 [ 129.          115.75776398   74.00621118]
 [   4.53526221    2.65280289    1.82459313]]
kmeans - cov = 
 [[[  91.93416154   -9.85273408   -1.89289965]
  [  -9.85273408 

In [ ]:
# for EM algorithm   
likelihood = []
iter_num = 10

def Cal_mean(image, means, gamma_mat):
    new_means = np.zeros((K_cluster,3))   
    
    for k in range(K_cluster):
        r = 0.0
        g = 0.0
        b = 0.0
        N_k = np.sum(gamma_mat[:,:,k])
        for i in range(im_w_re):
            for j in range(im_h_re):
                r += image[i,j,0]*gamma_mat[i,j,k]
                g += image[i,j,1]*gamma_mat[i,j,k]
                b += image[i,j,2]*gamma_mat[i,j,k]
        new_means[k,:] = np.array([r/N_k, g/N_k, b/N_k])
    return new_means

def Cal_cov(image, cov, means, gamma_mat):
    new_cov = np.zeros_like(cov)
    
    for k in range(K_cluster):
        val = new_cov[k,:,:]
        N_k = np.sum(gamma_mat[:,:,k])
        
        for i in range(im_w_re):
            for j in range(im_h_re):
                tmp = np.dot((image[i,j,:] - means[k,:]).reshape((3,1)), (image[i,j,:] - means[k,:]).reshape((1,3)))
                val += gamma_mat[i,j,k]*tmp
        new_cov[k,:,:] = val/N_k
    return new_cov

def Cal_pi(pis,gamma_mat):
    new_pis = np.zeros_like(pis)
    
    for k in range(K_cluster):
        N_k = np.sum(gamma_mat[:,:,k])
        new_pis[k] = N_k/float(im_w_re*im_h_re)
    return new_pis
    
def Cal_gamma(pixel, k, means, cov, pis):
    denom = 0.0
    gamma = 0
    nom = pis[k]*multivariate_normal.pdf(pixel, means[k,:], cov[k,:,:]+bias)
    for i in range(K_cluster):
        denom += pis[i]*multivariate_normal.pdf(pixel, means[i,:], cov[i,:,:]+bias)
    
    gamma = nom/denom
    return gamma

def EM_(image, means, covars, pis, k_array):
    
    # E step
    gamma_mat = np.zeros((im_w_re, im_h_re, K_cluster))
    for i in range(im_w_re):
        for j in range(im_h_re):     
            for k in range(K_cluster):
                gamma_mat[i,j,k] = Cal_gamma(image[i,j,:], k, means, covars, pis)
    
    # M step
    new_means = Cal_mean(image, means, gamma_mat)
    new_covars = Cal_cov(image, covars, new_means, gamma_mat)
    new_pis = Cal_pi(pis, gamma_mat)
    
    return gamma_mat,new_means, new_covars, new_pis

def log_likelihood(image, means, covars, pis):
    val = 0.0
    for i in range(im_w_re):
        for j in range(im_h_re):
            tmp = 0.0
            for k in range(K_cluster):
                # reshape mean to a vector
                mean_vec = means[k,:].reshape(-1)
                tmp += pis[k]*multivariate_normal.pdf(image[i,j,:], mean_vec, covars[k,:,:]+bias)
            val += np.log(tmp)
    return val

print('Start EM Algorithm')
for i in range(iter_num):
    print('iter = ',i)
    
    gamma_mat, means, cov, pi = EM_(image_resize, means, cov, pi, k_ary)
    
    print ('EM - means = \n',means)
    print ('EM - cov = \n',cov)
    print ('EMs - pi = \n',pi)
    print ('=========================================')
    
    likelihood.append(log_likelihood(image_resize, means, cov, pi))
    


Start EM Algorithm
iter =  0
EM - means = 
 [[  83.68631777   47.07439096    9.82555618]
 [ 106.28125898   96.21334781   17.18096974]
 [ 226.09836314   63.39656614    8.80273694]
 [ 233.46101985  202.17385048   37.5688176 ]
 [ 224.69165863  227.24031666  211.69562805]
 [ 209.87067688  191.12391458  154.69156381]
 [ 215.53691817  162.08021545   68.82731127]
 [  87.6184366    78.1633829    55.62516664]
 [ 170.60052325   52.22348673   13.0676705 ]
 [  49.98018359  112.70687026  204.91489071]
 [  54.71891561   40.98382656    3.46566204]
 [ 125.30540237   22.2153178     4.27288551]
 [ 164.43292102  134.82080439   45.94448117]
 [  43.72926706   42.09766243   35.60341173]
 [  64.3015281    10.02955671    1.77087843]
 [ 155.69126063  154.10020695  120.67030526]
 [  37.96695756   28.08557241    3.26951549]
 [  21.15494507   12.06729721    2.01872479]
 [ 129.54389073  116.11921589   71.22890903]
 [   4.33363627    2.36606862    1.49084697]]
EM - cov = 
 [[[ 100.19308172   -8.64840736   -4.675561

iter =  3
EM - means = 
 [[  81.55956445   43.88916152    8.25192039]
 [ 103.39336648   95.78376364   16.31727352]
 [ 227.1011538    61.84109101    7.0831216 ]
 [ 232.35088404  200.58003552   36.9037949 ]
 [ 225.19060046  227.70729934  212.43624829]
 [ 209.19811639  190.89448655  156.65590937]
 [ 218.62068751  162.00873137   69.67308239]
 [  90.66810544   77.15287865   52.09350462]
 [ 168.27200186   56.63456959   15.8779335 ]
 [  49.93504916  113.69319214  207.80143831]
 [  53.25937121   40.21968029    1.95523572]
 [ 129.31005551   20.29285999    3.07159886]
 [ 164.38364801  135.34849538   46.71510511]
 [  38.04551235   33.9049       24.23948225]
 [  51.26828113    6.99030622    0.47755969]
 [ 158.33228452  154.42030493  117.46038449]
 [  37.72441006   28.12187077    2.30106588]
 [  20.04494178   11.38433749    1.13794753]
 [ 130.40876789  116.90413853   67.50194857]
 [   3.33714997    1.70588057    0.90861508]]
EM - cov = 
 [[[  1.34525902e+02  -2.50766755e+00  -8.88499517e+00]
  [ -2

In [ ]:
print(means)

In [ ]:
# reconstruct image
def construct_image(image, means, gamma_matrixs):
    k_num = gamma_matrixs.shape[2]
    new_image = np.zeros_like(image)
    for i in range(im_w_re):
        for j in range(im_h_re):
            compare_ls = []
            for k in range(k_num):
                compare_ls.append(gamma_matrixs[i,j,k])
            min_idx = np.argmax(compare_ls)
            new_image[i,j,0] = means[min_idx,0]
            new_image[i,j,1] = means[min_idx,1]
            new_image[i,j,2] = means[min_idx,2]
    return new_image

new_image = construct_image(image_resize, means, gamma_mat)    

# plot log likelihood & image
plt.figure()
plt.plot(range(len(likelihood)),likelihood)
plt.title('log likelihood')

plt.figure()
plt.imshow(image_resize)
plt.title('resize image')

plt.figure()
plt.imshow(new_image)
plt.title('construct image')

# save fig
plt.savefig('./K_%s_reconstructImg.png'%str(K_cluster))

plt.show()

In [ ]:
# print(new_pis)
#now [0.51673384  0.48326616]

In [ ]:
# print(np.array([[1,2,3],[1,2,3]]).sum())

In [ ]:
# export to python file
#!jupyter nbconvert --to python Problem3_Gaussian_Mixture_Model.ipynb